# Initiation

In [1]:
import os
import random
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchmetrics import PeakSignalNoiseRatio as PSNR, StructuralSimilarityIndexMeasure as SSIM
from piq import LPIPS
import os
import torch.nn.functional as F
from torchvision.models import vgg19
from torchvision.utils import save_image
from torchvision.datasets import DatasetFolder
from datetime import datetime
from sklearn.model_selection import train_test_split
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
from copy import deepcopy

In [2]:
def set_seed(seed=42):
    """Set all random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed (choose any number you like)
set_seed(42)

# DataLoaders


In [3]:
class RandomAugment:
    def __init__(self):
        self.color_jitter = transforms.ColorJitter(
            contrast=0.05,
            saturation=0.1,
            hue=0.2
        )
        self.gaussian_blur = transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.5))
        # self.to_grayscale = transforms.RandomGrayscale(p=0.3)
        self.invert = transforms.RandomInvert(p=0.3)
        
    def __call__(self, img):
        # Apply augmentations in random order
        aug_order = random.sample(['color', 'blur', 'invert'], k=random.randint(0, 3))
        
        for aug in aug_order:
            if aug == 'color' and random.random() < 0.7:  # 70% chance
                img = self.color_jitter(img)
            elif aug == 'blur' and random.random() < 0.5:  # 50% chance
                img = self.gaussian_blur(img)
            elif aug == 'invert':
                img = self.invert(img)
                
        return img
    
def show_true_images(low_batch, high_batch, n=4):

    """
    Shows EXACTLY what's in your dataset with correct color handling.
    Works for both normalized and unnormalized images.
    """

    plt.figure(figsize=(18, 8))

    for i in range(min(n, len(low_batch))):
        # --- Low-light ---
        plt.subplot(2, n, i+1)
        low_img = low_batch[i].permute(1, 2, 0).numpy()

        # Handle normalization if present
        if low_img.min() < 0:  # Likely normalized
            low_img = (low_img * 0.5 + 0.5)  # Reverse imagenet norm

        # Ensure proper image range
        low_img = np.clip(low_img, 0, 1)
        plt.imshow(low_img)
        plt.title(f"Low")
        plt.axis('off')

        # --- Normal-light ---
        plt.subplot(2, n, n+i+1)
        high_img = high_batch[i].permute(1, 2, 0).numpy()

        if high_img.min() < 0:  # Likely normalized
            high_img = (high_img * 0.5 + 0.5)

        high_img = np.clip(high_img, 0, 1)
        plt.imshow(high_img)
        plt.title(f"High")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Run Data

In [4]:
# Your existing dataset code
dataset_path1 = r"/home/ahansviar2/Deep Learning Project (GAN for Light)/downloaded_images"
train_path = f'{dataset_path1}/train'
val_path = f'{dataset_path1}/val'
test_path = f'{dataset_path1}/test'

# Usage in Dataset
transform = transforms.Compose([
    # RandomAugment(),  # Your custom augmentations
    # transforms.GaussianBlur(kernel_size=3, sigma=(0.1,0.5)),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Scales to [-1, 1]
    transforms.Resize((256, 256)), 
    transforms.ToTensor(),
])

low_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

high_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
class CleanDataset(Dataset):
    def __init__(self, root_dir, low_transform=None, high_transform=None):
        self.root_dir = root_dir
        self.low_transform = low_transform
        self.high_transform = high_transform
        self.low_dir = os.path.join(root_dir, "low")
        self.high_dir = os.path.join(root_dir, "high")
        self.image_names = sorted(os.listdir(self.low_dir))  # Ensures matching pairs

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        low_img_path = os.path.join(self.low_dir, self.image_names[idx])
        high_img_path = os.path.join(self.high_dir, self.image_names[idx])

        low_img = Image.open(low_img_path).convert("RGB")
        high_img = Image.open(high_img_path).convert("RGB")

        # Apply separate transforms
        if self.low_transform:
            low_img = self.low_transform(low_img)
        if self.high_transform:
            high_img = self.high_transform(high_img)
            
        return low_img, high_img, os.path.basename(low_img_path)

# Initialize datasets ONCE with correct parameters
train_dataset = CleanDataset(
    root_dir=train_path, 
    low_transform=low_transform,
    high_transform=high_transform
)

val_dataset = CleanDataset(
    root_dir=val_path,
    low_transform=low_transform,
    high_transform=high_transform
)

test_dataset = CleanDataset(
    root_dir=test_path,
    low_transform=low_transform,
    high_transform=high_transform
)

# DataLoader setup (this part is correct)
batch_size = 8

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1
)

device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:5


## Data Checking

In [6]:
for split in ['train', 'val', 'test']:
    low_dir = os.path.join(dataset_path1, split, 'low')
    high_dir = os.path.join(dataset_path1, split, 'high')
    low_files = set(os.listdir(low_dir))
    high_files = set(os.listdir(high_dir))
    assert low_files == high_files, f"Mismatch in {split} set!"
print("✅ All datasets have matched low/high pairs.")

✅ All datasets have matched low/high pairs.


In [7]:
# Test one batch
low_batch, high_batch, _ = next(iter(train_loader))
print(f"Low batch shape: {low_batch.shape}")
print(f"High batch shape: {high_batch.shape}")
print(f"Pixel range - Low: [{low_batch.min():.2f}, {low_batch.max():.2f}]")
print(f"Pixel range - High: [{high_batch.min():.2f}, {high_batch.max():.2f}]")

Low batch shape: torch.Size([8, 3, 256, 256])
High batch shape: torch.Size([8, 3, 256, 256])
Pixel range - Low: [0.00, 1.00]
Pixel range - High: [0.00, 1.00]


In [8]:
def validate_image_pairs(dataset, n_samples=5):
    """Plot random pairs to visually inspect alignment/quality"""
    indices = random.sample(range(len(dataset)), n_samples)
    for idx in indices:
        low, high, _ = dataset[idx]
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(low.permute(1, 2, 0).clamp(0, 1))
        plt.title("Low-light")
        plt.subplot(1, 2, 2)
        plt.imshow(high.permute(1, 2, 0).clamp(0, 1))
        plt.title("Normal-light")
        plt.show()

# validate_image_pairs(train_dataset)

In [9]:
def analyze_channel_stats(loader):
    """Calculate mean/std per channel across dataset"""
    channels_sum, channels_sq_sum, num_batches = 0, 0, 0
    for low, high, _, in loader:
        channels_sum += torch.mean(low, dim=[0, 2, 3])
        channels_sq_sum += torch.mean(low**2, dim=[0, 2, 3])
        num_batches += 1
    mean = channels_sum / num_batches
    std = (channels_sq_sum / num_batches - mean**2)**0.5
    print(f"Low-light stats - Mean: {mean}, Std: {std}")

analyze_channel_stats(train_loader)

Low-light stats - Mean: tensor([0.1954, 0.1936, 0.1911]), Std: tensor([0.3196, 0.3206, 0.3242])


In [10]:
low, high, _ = next(iter(train_loader))
print(f"Batch memory: {low.element_size() * low.nelement() / 1024**2:.2f} MB")

Batch memory: 6.00 MB


# MODEL ARCHITECTURE

## Diffusion Experiment 2

In [11]:
import math
import os
import time
import torch
import torch.nn.functional as F
from torchmetrics import StructuralSimilarityIndexMeasure as SSIM
from piq import LPIPS
from tqdm import tqdm
from torch import nn, optim

class TimeEmbedding(nn.Module):
    def __init__(self, dim, device):
        super().__init__()
        self.dim = dim
        half_dim = dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        self.register_buffer('emb', emb)

    def forward(self, t):
        emb = t[:, None] * self.emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

class UNetBlock(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim):
        super().__init__()
        self.time_mlp = nn.Linear(time_emb_dim, out_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.norm = nn.BatchNorm2d(out_ch)
        self.act = nn.SiLU()

    def forward(self, x, t):
        h = self.conv1(x)
        time_emb = self.act(self.time_mlp(t))
        h = h + time_emb[:, :, None, None]
        h = self.norm(h)
        h = self.act(h)
        h = self.conv2(h)
        h = self.norm(h)
        h = self.act(h)
        return h
    
class UNet(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.time_mlp = nn.Sequential(
            TimeEmbedding(32, device),
            nn.Linear(32, 32),
            nn.SiLU(),
            nn.Linear(32, 32)
        )
        
        # Down blocks
        self.down1 = UNetBlock(3, 64, 32)
        self.down2 = UNetBlock(64, 128, 32)
        self.down3 = UNetBlock(128, 256, 32)
        
        # Bottleneck
        self.bottleneck = UNetBlock(256, 512, 32)
        
        # Up blocks
        self.up1 = UNetBlock(512 + 256, 256, 32)
        self.up2 = UNetBlock(256 + 128, 128, 32)
        self.up3 = UNetBlock(128 + 64, 64, 32)
        
        # Output
        self.out = nn.Conv2d(64, 3, 1)

    def forward(self, x, t):
        t = self.time_mlp(t)
        d1 = self.down1(x, t)
        d2 = self.down2(F.max_pool2d(d1, 2), t)
        d3 = self.down3(F.max_pool2d(d2, 2), t)
        
        h = F.max_pool2d(d3, 2)
        h = self.bottleneck(h, t)
        
        h = F.interpolate(h, scale_factor=2)
        h = torch.cat([h, d3], dim=1)
        h = self.up1(h, t)
        
        h = F.interpolate(h, scale_factor=2)
        h = torch.cat([h, d2], dim=1)
        h = self.up2(h, t)
        
        h = F.interpolate(h, scale_factor=2)
        h = torch.cat([h, d1], dim=1)
        h = self.up3(h, t)
        
        return self.out(h)

class FrequencyCompensatedDecoder(nn.Module):
    def __init__(self, latent_dim=256, output_ch=3):
        super().__init__()
        # Input projection to match latent_dim
        self.input_proj = nn.Conv2d(3, latent_dim, 1)  # Changed from 259 to 3
        
        # LR feature extractor
        self.lr_feature_extractor = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.GroupNorm(32, 64),
            nn.SiLU(),
            nn.Conv2d(64, latent_dim, 3, padding=1)
        )
        
        # Feature fusion
        self.fusion = nn.Sequential(
            nn.Conv2d(latent_dim*2, latent_dim, 1),
            nn.GroupNorm(32, latent_dim),
            nn.SiLU()
        )
        
        # Frequency blocks
        self.aff_blocks = nn.ModuleList([
            AFFBlock(latent_dim) for _ in range(6)
        ])
        
        # Output
        self.output_conv = nn.Sequential(
            nn.Conv2d(latent_dim, 64, 3, padding=1),
            nn.GroupNorm(32, 64),
            nn.SiLU(),
            nn.Conv2d(64, output_ch, 3, padding=1)
        )

    def forward(self, x, lr_condition):
        # Project input to latent_dim (3 -> 256)
        x = self.input_proj(x)
        
        # Extract LR features (3 -> 256)
        lr_features = self.lr_feature_extractor(lr_condition)
        
        # Fuse features (256 + 256 -> 256)
        x = self.fusion(torch.cat([x, lr_features], dim=1))
        
        # Frequency processing
        for block in self.aff_blocks:
            x = block(x)
            
        return self.output_conv(x)

class AFFBlock(nn.Module):
    """Adaptive Frequency Filtering block with proper FFT handling"""
    def __init__(self, channels):
        super().__init__()
        self.channels = channels
        self.conv = nn.Conv2d(channels, channels, 3, padding=1)
        self.norm = nn.GroupNorm(32, channels)
        self.act = nn.SiLU()
        
        # Learnable frequency filter
        self.filter = nn.Parameter(torch.ones(1, channels, 1, 1))
        
    def forward(self, x):
        # Spatial processing
        h = self.act(self.norm(self.conv(x)))
        
        # Frequency processing
        fft = torch.fft.rfft2(h, norm='ortho')
        mag = torch.abs(fft)
        phase = torch.angle(fft)
        
        # Apply learned frequency filter
        filtered = mag * self.filter
        
        # Inverse FFT
        real = filtered * torch.cos(phase)
        imag = filtered * torch.sin(phase)
        complex_tensor = torch.complex(real, imag)
        reconstructed = torch.fft.irfft2(complex_tensor, norm='ortho')
        
        return x + reconstructed

# Update SSMoE_UNet to handle input_channels
class SSMoE_UNet(nn.Module):
    def __init__(self, device, num_experts=4, T=1000, in_channels=3):
        super().__init__()
        self.device = device
        self.num_experts = num_experts
        self.T = T
        self.time_embed_dim = 32
        
        # Time embedding
        self.time_mlp = nn.Sequential(
            TimeEmbedding(self.time_embed_dim, device),
            nn.Linear(self.time_embed_dim, self.time_embed_dim),
            nn.SiLU(),
            nn.Linear(self.time_embed_dim, self.time_embed_dim)
        )
        
        # Down blocks
        self.down1 = UNetBlock(in_channels, 64, self.time_embed_dim)
        self.down2 = UNetBlock(64, 128, self.time_embed_dim)
        self.down3 = UNetBlock(128, 256, self.time_embed_dim)
        
        # Bottleneck with space MoE
        self.bottleneck = UNetBlock(256, 512, self.time_embed_dim)
        
        # Up blocks
        self.up1 = UNetBlock(512 + 256, 256, self.time_embed_dim)
        self.up2 = UNetBlock(256 + 128, 128, self.time_embed_dim)
        self.up3 = UNetBlock(128 + 64, 64, self.time_embed_dim)
        
        # Output
        self.out = nn.Conv2d(64, 3, 1)
        
        # Initialize sampling MoEs
        self.sampling_experts = nn.ModuleList([
            self._create_expert() for _ in range(num_experts)
        ])
        
    def _create_expert(self):
        """Create one expert network"""
        return nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.GroupNorm(32, 64),
            nn.SiLU(),
            nn.Conv2d(64, 3, 3, padding=1)
        )
    
    def forward(self, x, t):
        # Time embedding
        t_emb = self.time_mlp(t)
        
        # Select expert based on timestep
        stage = (t.float() / self.T * self.num_experts).long().clamp(0, self.num_experts-1)
        expert_out = torch.zeros_like(x, device=self.device)
        
        # Process each expert batch separately
        for i in range(self.num_experts):
            mask = (stage == i)
            if mask.any():
                expert_out[mask] = self.sampling_experts[i](x[mask])
        
        # Downsample
        d1 = self.down1(x + expert_out, t_emb)
        d2 = self.down2(F.max_pool2d(d1, 2), t_emb)
        d3 = self.down3(F.max_pool2d(d2, 2), t_emb)
        
        # Bottleneck
        h = F.max_pool2d(d3, 2)
        h = self.bottleneck(h, t_emb)
        
        # Upsample with skip connections
        h = F.interpolate(h, scale_factor=2)
        h = torch.cat([h, d3], dim=1)
        h = self.up1(h, t_emb)
        
        h = F.interpolate(h, scale_factor=2)
        h = torch.cat([h, d2], dim=1)
        h = self.up2(h, t_emb)
        
        h = F.interpolate(h, scale_factor=2)
        h = torch.cat([h, d1], dim=1)
        h = self.up3(h, t_emb)
        
        return self.out(h)

class LatentDiffusionModel:
    def __init__(self, T=1000, beta_start=1e-4, beta_end=0.02, device='cuda'):
        self.T = T
        self.device = device
        self.betas = torch.linspace(beta_start, beta_end, T, device=device)
        self.alphas = 1. - self.betas
        self.alpha_bars = torch.cumprod(self.alphas, dim=0)
        
        # Initialize components with proper T
        self.denoise_unet = SSMoE_UNet(device, T=T).to(device)
        self.decoder = FrequencyCompensatedDecoder().to(device)
        
        # Metrics
        self.lpips = LPIPS(replace_pooling=True).to(device).eval()
        self.ssim = SSIM().to(device).eval()
        
        # Tracking training progress
        self.current_epoch = 0
        self.total_epochs = 200
    
    def forward_diffuse(self, x0, t):
        """Forward diffusion with noise schedule"""
        t = t.to(self.device)
        noise = torch.randn_like(x0)
        sqrt_alpha_bar = torch.sqrt(self.alpha_bars[t])[:, None, None, None]
        sqrt_one_minus_alpha_bar = torch.sqrt(1. - self.alpha_bars[t])[:, None, None, None]
        xt = sqrt_alpha_bar * x0 + sqrt_one_minus_alpha_bar * noise
        return xt, noise
    
    def train_step(self, x0, lr_condition, optimizer):
        optimizer.zero_grad()
        
        # Sample random timesteps
        t = torch.randint(0, self.T, (x0.size(0),), device=self.device)
        
        # Forward diffusion
        xt, noise = self.forward_diffuse(x0, t)
        
        # Predict noise with UNet
        pred_noise = self.denoise_unet(xt, t)
        
        # Decode to pixel space
        pred_img = self.decoder(xt - pred_noise, lr_condition)
        
        # Hybrid loss
        loss, metrics = self.hybrid_loss(pred_img, x0, pred_noise, noise)
        
        loss.backward()
        optimizer.step()
        return metrics
    
    def hybrid_loss(self, pred, target, noise_pred, true_noise):
        """Combined loss with dynamic weighting"""
        # Base losses
        mse_loss = F.mse_loss(noise_pred, true_noise)
        lpips_loss = self.lpips(pred, target)
        ssim_loss = 1 - self.ssim(pred, target)
        
        # Dynamic weights (progressively focus more on perceptual quality)
        progress = self.current_epoch / self.total_epochs
        lpips_w = 0.4  # Fixed high importance for perceptual quality
        ssim_w = 0.3 * progress  # Increasing structural importance
        mse_w = 1.0 - lpips_w - ssim_w  # Decreasing noise prediction importance
        
        total_loss = mse_w * mse_loss + lpips_w * lpips_loss + ssim_w * ssim_loss
        
        # Additional metrics
        with torch.no_grad():
            psnr = 10 * torch.log10(1 / F.mse_loss(pred, target))
            
        return total_loss, {
            'loss': total_loss.item(),
            'mse': mse_loss.item(),
            'lpips': lpips_loss.item(),
            'ssim': 1 - ssim_loss.item(),
            'psnr': psnr.item()
        }

# Setup device
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Ensure checkpoint directory exists
os.makedirs('checkpoints_diffusion_nonorm', exist_ok=True)

# Initialize model and optimizer
diffusion = LatentDiffusionModel(device=device)
optimizer = optim.Adam(
    list(diffusion.denoise_unet.parameters()) + 
    list(diffusion.decoder.parameters()),
    lr=1e-4
)

# Training loop
epochs = 200
diffusion.total_epochs = epochs
best_metrics = {'loss': float('inf'), 'psnr': 0, 'ssim': 0, 'lpips': float('inf')}

# Record total training time
start_time = time.time()

for epoch in range(epochs):
    diffusion.current_epoch = epoch
    epoch_metrics = {'loss': 0, 'mse': 0, 'lpips': 0, 'ssim': 0, 'psnr': 0}
    
    for low_imgs, high_imgs, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        low_imgs, high_imgs = low_imgs.to(device), high_imgs.to(device)
        
        # Train step with LR conditioning
        metrics = diffusion.train_step(high_imgs, low_imgs, optimizer)
        
        # Update metrics
        for k in epoch_metrics:
            epoch_metrics[k] += metrics[k]
    
    # Average metrics
    for k in epoch_metrics:
        epoch_metrics[k] /= len(train_loader)
    
    # Save every epoch
    torch.save({
        'epoch': epoch,
        'denoise_unet': diffusion.denoise_unet.state_dict(),
        'decoder': diffusion.decoder.state_dict(),
        'optimizer': optimizer.state_dict(),
        'metrics': epoch_metrics
    }, f'checkpoints_diffusion_nonorm/diffusion_epoch_{epoch+1}.pth')
    
    # Save best model
    if epoch_metrics['loss'] < best_metrics['loss']:
        best_metrics = epoch_metrics
        torch.save({
            'epoch': epoch,
            'denoise_unet': diffusion.denoise_unet.state_dict(),
            'decoder': diffusion.decoder.state_dict(),
            'optimizer': optimizer.state_dict(),
            'metrics': epoch_metrics
        }, 'checkpoints_diffusion_nonorm/diffusion_best_model.pth')
    
    print(f"Epoch {epoch+1} | "
          f"Loss: {epoch_metrics['loss']:.4f} | "
          f"PSNR: {epoch_metrics['psnr']:.2f} dB | "
          f"SSIM: {epoch_metrics['ssim']:.3f} | "
          f"LPIPS: {epoch_metrics['lpips']:.3f}")

total_training_time = time.time() - start_time
hours, remainder = divmod(total_training_time, 3600)
minutes, seconds = divmod(remainder, 60)

print(f"\nTraining complete! Best metrics: "
      f"Loss: {best_metrics['loss']:.4f}, "
      f"PSNR: {best_metrics['psnr']:.2f} dB, "
      f"SSIM: {best_metrics['ssim']:.3f}, "
      f"LPIPS: {best_metrics['lpips']:.3f}")
print(f"Total training time: {int(hours)}h {int(minutes)}m {seconds:.2f}s")

# Save final model
torch.save({
    'epoch': epochs,
    'denoise_unet': diffusion.denoise_unet.state_dict(),
    'decoder': diffusion.decoder.state_dict(),
    'optimizer': optimizer.state_dict(),
    'metrics': epoch_metrics,
    'training_time': total_training_time
}, 'checkpoints_diffusion_nonorm/diffusion_final_model.pth')

/home/ahansviar2/Deep Learning Project (GAN for Light)/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ahansviar2/Deep Learning Project (GAN for Light)/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ahansviar2/Deep Learning Project (GAN for Light)/.venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndex

Epoch 1 | Loss: 0.4660 | PSNR: 12.68 dB | SSIM: 0.606 | LPIPS: 0.439


Epoch 2/200: 100%|██████████| 49/49 [00:47<00:00,  1.03it/s]


Epoch 2 | Loss: 0.2007 | PSNR: 15.65 dB | SSIM: 0.781 | LPIPS: 0.276


Epoch 3/200: 100%|██████████| 49/49 [00:47<00:00,  1.03it/s]


...

Epoch 198 | Loss: 0.0345 | PSNR: 15.57 dB | SSIM: 0.998 | LPIPS: 0.076


Epoch 199/200: 100%|██████████| 49/49 [00:47<00:00,  1.03it/s]


Epoch 199 | Loss: 0.0350 | PSNR: 15.49 dB | SSIM: 0.998 | LPIPS: 0.079


Epoch 200/200: 100%|██████████| 49/49 [00:47<00:00,  1.03it/s]


Epoch 200 | Loss: 0.0340 | PSNR: 15.29 dB | SSIM: 0.998 | LPIPS: 0.075

Training complete! Best metrics: Loss: 0.0330, PSNR: 15.31 dB, SSIM: 0.998, LPIPS: 0.075
Total training time: 2h 46m 13.89s


In [21]:
import os
import torch
from torchvision import transforms
from PIL import ImageDraw, ImageFont
from tqdm import tqdm

def evaluate_diffusion(denoise_unet, decoder, test_loader, device, diffusion, save_samples=True, sample_dir="sample"):
    # Initialize metrics
    psnr = PSNR().to(device)
    ssim = SSIM().to(device)
    lpips = LPIPS(replace_pooling=True).to(device)
    
    metrics = {
        'per_image': [],
        'average': {
            'psnr': 0.0,
            'ssim': 0.0,
            'lpips': 0.0
        }
    }
    
    if save_samples:
        os.makedirs(sample_dir, exist_ok=True)
    
    denoise_unet.eval()
    decoder.eval()
    sample_counter = 0
    
    with torch.no_grad():
        for batch_idx, (low_imgs, high_imgs, *_) in enumerate(tqdm(test_loader, desc="Evaluating")):
            low_imgs, high_imgs = low_imgs.to(device), high_imgs.to(device)
            
            # Use t=0 (final timestep) for evaluation
            t = torch.zeros(low_imgs.size(0), dtype=torch.long, device=device)
            
            # Forward diffusion process
            xt, _ = diffusion.forward_diffuse(high_imgs, t)
            
            # Denoise and decode
            pred_noise = denoise_unet(xt, t)
            enhanced_imgs = decoder(xt - pred_noise, low_imgs)
            
            # Clamp outputs to valid range
            enhanced_imgs = enhanced_imgs.clamp(0, 1)
            high_imgs = high_imgs.clamp(0, 1)
            
            for img_idx in range(low_imgs.size(0)):
                # Calculate metrics
                img_metrics = {
                    'psnr': psnr(enhanced_imgs[img_idx].unsqueeze(0), high_imgs[img_idx].unsqueeze(0)).item(),
                    'ssim': ssim(enhanced_imgs[img_idx].unsqueeze(0), high_imgs[img_idx].unsqueeze(0)).item(),
                    'lpips': lpips(enhanced_imgs[img_idx].unsqueeze(0), high_imgs[img_idx].unsqueeze(0)).item()
                }
                
                # Denormalize low-light image for visualization
                low_img_denorm = (low_imgs[img_idx] * 0.5) + 0.5
                low_img_denorm = low_img_denorm.clamp(0, 1)
                
                # Create a 4-column comparison image (C, H, W*4)
                comparison = torch.cat([
                    low_imgs[img_idx].clamp(0, 1),  # Low-light (normalized)
                    low_img_denorm,                 # Low-light (denormalized)
                    enhanced_imgs[img_idx],         # Enhanced
                    high_imgs[img_idx]              # Ground truth
                ], dim=-1)
                
                # Convert to PIL
                comparison_pil = transforms.ToPILImage()(comparison.cpu())
                
                if save_samples:
                    # Add labels
                    draw = ImageDraw.Draw(comparison_pil)
                    try:
                        font = ImageFont.truetype("arial.ttf", 15)
                    except:
                        font = ImageFont.load_default()
                    
                    width = comparison_pil.width // 4
                    draw.text((10, 10), "Low Light\n(Normalized)", fill="white", font=font)
                    draw.text((width + 10, 10), "Low Light\n(Denormalized)", fill="white", font=font)
                    draw.text((2 * width + 10, 10), "Enhanced", fill="white", font=font)
                    draw.text((3 * width + 10, 10), "Ground Truth", fill="white", font=font)

                    # Metrics under Enhanced
                    text_y = comparison_pil.height - 40
                    draw.text((2 * width + 10, text_y), 
                              f"PSNR: {img_metrics['psnr']:.2f}\nSSIM: {img_metrics['ssim']:.3f}\nLPIPS: {img_metrics['lpips']:.3f}", 
                              fill="white", font=font)

                    # Save image
                    sample_path = os.path.join(sample_dir, f"sample_{sample_counter:04d}.png")
                    comparison_pil.save(sample_path)
                    img_metrics['image_path'] = sample_path
                    sample_counter += 1
                
                # Store metrics
                metrics['per_image'].append(img_metrics)
                metrics['average']['psnr'] += img_metrics['psnr']
                metrics['average']['ssim'] += img_metrics['ssim']
                metrics['average']['lpips'] += img_metrics['lpips']
    
    # Compute averages
    total_samples = len(metrics['per_image'])
    metrics['average']['psnr'] /= total_samples
    metrics['average']['ssim'] /= total_samples
    metrics['average']['lpips'] /= total_samples
    
    print(f"\nEvaluation Results (Averages):")
    print(f"PSNR: {metrics['average']['psnr']:.2f} dB")
    print(f"SSIM: {metrics['average']['ssim']:.4f}")
    print(f"LPIPS: {metrics['average']['lpips']:.4f}")
    
    return metrics

# Initialize model
diffusion = LatentDiffusionModel(device=device)

# Load checkpoint
checkpoint = torch.load('checkpoints_diffusion_nonorm/diffusion_epoch_39.pth', map_location=device)

# Load state_dict into model components
diffusion.denoise_unet.load_state_dict(checkpoint['denoise_unet'])
diffusion.decoder.load_state_dict(checkpoint['decoder'])

# Evaluate
metrics = evaluate_diffusion(
    denoise_unet=diffusion.denoise_unet,
    decoder=diffusion.decoder,
    test_loader=test_loader,  # Your test DataLoader
    device=device,
    diffusion=diffusion,      # Already has forward_diffuse()
    save_samples=True,         # Save all comparison images
    sample_dir="diffusion_nonorm_sample"
)

Evaluating: 100%|██████████| 15/15 [00:02<00:00,  6.27it/s]


Evaluation Results (Averages):
PSNR: 20.18 dB
SSIM: 0.8569
LPIPS: 0.1440
